### Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import ClassificationMetric
import matplotlib.pyplot as plt

### Load the CSV File

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/mnt/data/Student Feedback form_IDA1_CS1202 - Student Feedback form_IDA1_CS1202.csv.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())

# Check for missing values
print(df.isnull().sum())

# Basic description of the dataset
print(df.describe())


### Pre-process the Data

In [ ]:
# Fill missing values if any
df.fillna(method='ffill', inplace=True)

# Convert categorical 'Gender' column to numerical representation (0 for Male, 1 for Female)
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})


#### Define the Target and Feature

In [ ]:
# Define features (X) and target (y)
X = df.drop(columns=['Feedback Rating'])  # Drop target column
y = df['Feedback Rating']                 # Define the target

# Sensitive feature (Gender)
sensitive_feature = 'Gender'


### Convert the Dataset into AIF360 Format

In [ ]:
from aif360.datasets import BinaryLabelDataset

# Convert the dataset into AIF360 BinaryLabelDataset format
data = BinaryLabelDataset(df=pd.concat([X, y], axis=1),
                          label_names=['Feedback Rating'],
                          protected_attribute_names=['Gender'])


### Split the Data into Train and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train, test = data.split([0.8], shuffle=True)


### Pre Processing: Reweighing the Dataset to Mitigate Bias

In [ ]:
reweigher = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_reweighed = reweigher.fit_transform(train)

### In Processing: Adversarial Debiasing Model

In [ ]:
import tensorflow as tf
from aif360.algorithms.inprocessing import AdversarialDebiasing


In [ ]:
sess = tf.Session()

#### Set Up and Train the Adversarial Debiasing Model:

In [ ]:
# Set up the Adversarial Debiasing model
sess = tf.Session()

# Create the AdversarialDebiasing model
adv_debiasing = AdversarialDebiasing(privileged_groups=[{'Gender': 1}],   # Privileged group: Female
                                     unprivileged_groups=[{'Gender': 0}],  # Unprivileged group: Male
                                     scope_name='debiasing_classifier',
                                     debias=True,                         # Enable debiasing
                                     sess=sess)

#### Train the model

In [ ]:
# Train the model
adv_debiasing.fit(train)

### Make Predictions with the Trained Model

In [ ]:
# Predict on test data
predictions = adv_debiasing.predict(test)


### Evaluate the Model's Fairness and Performance after In Processing

#### Performance Evaluation - Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
test_labels = test.labels
pred_labels = predictions.labels

accuracy = accuracy_score(test_labels, pred_labels)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
# Accuracy
accuracy_pre = accuracy_score(test.labels, predictions.labels)
print(f"Test Accuracy (Pre Post-processing): {accuracy_pre:.4f}")

#### Fairness Evaluation

In [ ]:
from aif360.metrics import ClassificationMetric

# Compute fairness metrics
metric = ClassificationMetric(test, predictions,
                              unprivileged_groups=[{'Gender': 0}],
                              privileged_groups=[{'Gender': 1}])

#### Fairness: Disparate Impact and Equal Opportunity Difference

In [ ]:

disparate_impact_pre = metric_pre.disparate_impact()
equal_opportunity_pre = metric_pre.equal_opportunity_difference()

print(f"Disparate Impact (Pre Post-processing): {disparate_impact_pre}")
print(f"Equal Opportunity Difference (Pre Post-processing): {equal_opportunity_pre}")

In [ ]:
# Demographic parity
print(f"Demographic Parity Difference: {metric.disparate_impact()}")

# Equal opportunity difference
print(f"Equal Opportunity Difference: {metric.equal_opportunity_difference()}")

### Post Processing - Equalized Odds Post-processing

In [ ]:
eq_odds = EqOddsPostprocessing(privileged_groups=[{'Gender': 1}], 
                               unprivileged_groups=[{'Gender': 0}])

predictions_post = eq_odds.fit_predict(test, predictions)

### Evaluate Performance and Fairness After Post-processing

#### Performance: Accuracy

In [ ]:

accuracy_post = accuracy_score(test.labels, predictions_post.labels)
print(f"Test Accuracy (Post Post-processing): {accuracy_post:.4f}")

#### Fairness Metrics

In [ ]:

metric_post = ClassificationMetric(test, predictions_post,
                                   unprivileged_groups=[{'Gender': 0}],
                                   privileged_groups=[{'Gender': 1}])

#### Fairness: Disparate Impact and Equal Opportunity Difference

In [ ]:
disparate_impact_post = metric_post.disparate_impact()
equal_opportunity_post = metric_post.equal_opportunity_difference()

print(f"Disparate Impact (Post Post-processing): {disparate_impact_post}")
print(f"Equal Opportunity Difference (Post Post-processing): {equal_opportunity_post}")

### Visualizing Bias Mitigation

#### Plot Demographic Parity (Disparate Impact)

In [ ]:
import matplotlib.pyplot as plt

# Plot demographic parity difference
plt.bar(['Demographic Parity Difference'], [metric.disparate_impact()])
plt.title('Demographic Parity Difference')
plt.show()

# Plot equal opportunity difference
plt.bar(['Equal Opportunity Difference'], [metric.equal_opportunity_difference()])
plt.title('Equal Opportunity Difference')
plt.show()


#### Plot Equal Opportunity Difference

In [ ]:
plt.bar(['Pre Post-processing', 'Post Post-processing'], [equal_opportunity_pre, equal_opportunity_post])
plt.title('Equal Opportunity Difference')
plt.ylabel('Equal Opportunity Difference')
plt.show()